In [21]:
import pandas as pd
import numpy as np
from pandasql import *
import duckdb

In [22]:
duck_query = lambda q: duckdb.query(q)
pysqldf = lambda q: sqldf(q,globals())

## ----------------------------- Clase 4: Ejercicio -----------------------------

#### **Cafetería Peet's Coffee** 
Usted ha sido contratado por una cafetería que cuenta con tiendas en NY, USA. Algunos de sus productos inlcuyen bebidas de café y té, pastelería, entre otros.

Actualmente la cafetería no cuenta con reportería interna por lo que le han solicitado generar un reporte diario de las distintas ordenes vendidas.

Para ello, usted deberá de crear una única tabla que corra todos los días para que el reporte se actualize automáticamente. Este reporte deberá de llevar la siguiente información:
* Fecha
* Punto de venta
* Categoría
* Cantidad de ordenes
* Venta total (monto)


#### **Instrucciones:**
1. Genere una tabla con la información requerida que incluya los meses de Enero hasta Mayo 2023.
2. Genere un query que elimine los últimos 10 días e inserte los registros hasta la fecha.
3. Corra el query #2 las veces que sean necesarias para que toda la información se encuentre dentro del reporte.
4. Valide que el reporte cuente con todas las fechas de la tabla original. (funciones MIN(), MAX())

In [23]:
df = pd.read_csv("Coffee_Sales_Orders_Peets.csv")

In [24]:
df.head()

,transaction_id,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail,order_id,transaction_dt,transaction_ts
0,1,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg,613141,2023-01-01,2023-01-01 07:06:11
1,2,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,168935,2023-01-01,2023-01-01 07:08:56
2,3,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg,301867,2023-01-01,2023-01-01 07:14:04
3,4,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm,593276,2023-01-01,2023-01-01 07:20:24
4,5,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,654074,2023-01-01,2023-01-01 07:22:41


In [31]:
duck_query(
    """
    CREATE OR REPLACE TABLE coffee_sales_orders AS
    SELECT 
        * EXCLUDE(transaction_dt, transaction_ts)
        ,CAST(transaction_dt AS DATE) transaction_dt
        ,CAST(transaction_ts AS DATETIME) transaction_ts
    FROM df
    """
)

In [32]:
duck_query(
    """
    SELECT *
    FROM coffee_sales_orders
    LIMIT 20
    """
)

┌────────────────┬─────────────────┬──────────┬─────────────────┬───┬──────────┬────────────────┬─────────────────────┐
│ transaction_id │ transaction_qty │ store_id │ store_location  │ … │ order_id │ transaction_dt │   transaction_ts    │
│     int64      │      int64      │  int64   │     varchar     │   │  int64   │      date      │      timestamp      │
├────────────────┼─────────────────┼──────────┼─────────────────┼───┼──────────┼────────────────┼─────────────────────┤
│              1 │               2 │        5 │ Lower Manhattan │ … │   613141 │ 2023-01-01     │ 2023-01-01 07:06:11 │
│              2 │               2 │        5 │ Lower Manhattan │ … │   168935 │ 2023-01-01     │ 2023-01-01 07:08:56 │
│              3 │               2 │        5 │ Lower Manhattan │ … │   301867 │ 2023-01-01     │ 2023-01-01 07:14:04 │
│              4 │               1 │        5 │ Lower Manhattan │ … │   593276 │ 2023-01-01     │ 2023-01-01 07:20:24 │
│              5 │               2 │    

In [ ]:
3687 en el count all fecha final 30 de junio 

In [42]:
con.execute("""
    CREATE OR REPLACE TABLE coffee_sales_report_dly AS
    SELECT
        transaction_dt,
        store_location,
        product_category,
        COUNT(DISTINCT order_id) AS n_orders,
        ROUND(SUM(transaction_qty * unit_price), 2) AS total_amount
    FROM
        coffee_sales_orders
    WHERE
        transaction_dt BETWEEN '2023-01-01' AND '2023-05-30'
    GROUP BY 1, 2, 3
    ORDER BY 1, 2, 3
""")
result = con.execute("SELECT * FROM coffee_sales_report_dly LIMIT 20").fetchdf()
print(result)

   transaction_dt   store_location    product_category  n_orders  total_amount
0      2023-01-01          Astoria              Bakery        17         61.00
1      2023-01-01          Astoria              Coffee        82        381.80
2      2023-01-01          Astoria  Drinking Chocolate        25        142.50
3      2023-01-01          Astoria                 Tea        66        283.10
4      2023-01-01   Hell's Kitchen              Bakery        18         65.25
5      2023-01-01   Hell's Kitchen              Coffee        87        427.05
6      2023-01-01   Hell's Kitchen  Drinking Chocolate        13         71.75
7      2023-01-01   Hell's Kitchen                 Tea        66        287.40
8      2023-01-01  Lower Manhattan              Bakery        33        115.25
9      2023-01-01  Lower Manhattan              Coffee        72        351.25
10     2023-01-01  Lower Manhattan  Drinking Chocolate        12         81.25
11     2023-01-01  Lower Manhattan                 T

In [45]:
# Función para eliminar los últimos 10 días de registros
def delete_last_10_days():
    con.execute("""
        DELETE FROM coffee_sales_report_dly
        WHERE transaction_dt >= (
            SELECT MAX(transaction_dt) FROM coffee_sales_report_dly
        ) - INTERVAL '10 days'
    """)

# Llamar a la función para eliminar los últimos 10 días
delete_last_10_days()

# Verificar las primeras filas de la tabla coffee_sales_report_dly después de la eliminación
result_after_delete = con.execute("SELECT * FROM coffee_sales_report_dly LIMIT 20").fetchdf()
print(result_after_delete)

   transaction_dt   store_location    product_category  n_orders  total_amount
0      2023-01-01          Astoria              Bakery        17         61.00
1      2023-01-01          Astoria              Coffee        82        381.80
2      2023-01-01          Astoria  Drinking Chocolate        25        142.50
3      2023-01-01          Astoria                 Tea        66        283.10
4      2023-01-01   Hell's Kitchen              Bakery        18         65.25
5      2023-01-01   Hell's Kitchen              Coffee        87        427.05
6      2023-01-01   Hell's Kitchen  Drinking Chocolate        13         71.75
7      2023-01-01   Hell's Kitchen                 Tea        66        287.40
8      2023-01-01  Lower Manhattan              Bakery        33        115.25
9      2023-01-01  Lower Manhattan              Coffee        72        351.25
10     2023-01-01  Lower Manhattan  Drinking Chocolate        12         81.25
11     2023-01-01  Lower Manhattan                 T

In [46]:
# Función para eliminar los últimos 10 días de registros
def delete_last_10_days():
    con.execute("""
        DELETE FROM coffee_sales_report_dly
        WHERE transaction_dt >= (
            SELECT MAX(transaction_dt) FROM coffee_sales_report_dly
        ) - INTERVAL '10 days'
    """)

# Llamar a la función para eliminar los últimos 10 días
delete_last_10_days()

# Verificar las primeras filas de la tabla coffee_sales_report_dly después de la eliminación
result_after_delete = con.execute("SELECT * FROM coffee_sales_report_dly LIMIT 20").fetchdf()
print(result_after_delete)

   transaction_dt   store_location    product_category  n_orders  total_amount
0      2023-01-01          Astoria              Bakery        17         61.00
1      2023-01-01          Astoria              Coffee        82        381.80
2      2023-01-01          Astoria  Drinking Chocolate        25        142.50
3      2023-01-01          Astoria                 Tea        66        283.10
4      2023-01-01   Hell's Kitchen              Bakery        18         65.25
5      2023-01-01   Hell's Kitchen              Coffee        87        427.05
6      2023-01-01   Hell's Kitchen  Drinking Chocolate        13         71.75
7      2023-01-01   Hell's Kitchen                 Tea        66        287.40
8      2023-01-01  Lower Manhattan              Bakery        33        115.25
9      2023-01-01  Lower Manhattan              Coffee        72        351.25
10     2023-01-01  Lower Manhattan  Drinking Chocolate        12         81.25
11     2023-01-01  Lower Manhattan                 T

In [49]:
def validate_report_dates():
    # Obtener las fechas mínimas y máximas de la tabla original
    original_dates = con.execute("""
        SELECT 
            MIN(transaction_dt) AS min_order_date,
            MAX(transaction_dt) AS max_order_date
        FROM coffee_sales_orders
    """).fetchdf()
    
    # Obtener las fechas mínimas y máximas de la tabla del reporte
    report_dates = con.execute("""
        SELECT 
            MIN(transaction_dt) AS min_report_date,
            MAX(transaction_dt) AS max_report_date
        FROM coffee_sales_report_dly
    """).fetchdf()
    
    print("Original - Min y Max fechas:", original_dates)
    print("Reporte - Min y Max fechas:", report_dates)

# Validar que el reporte contenga todas las fechas de la tabla original
validate_report_dates()

Original - Min y Max fechas:   min_order_date max_order_date
0     2023-01-01     2023-06-30
Reporte - Min y Max fechas:   min_report_date max_report_date
0      2023-01-01      2023-05-08
